In [68]:
import logging
import os
#from _helpers import configure_logging

import pypsa
import yaml
import pandas as pd
import geopandas as gpd
import numpy as np
import scipy as sp
import networkx as nx
import matplotlib as plt

from scipy.sparse import csgraph
from itertools import product

from shapely.geometry import Point, LineString
import shapely, shapely.prepared, shapely.wkt

logger = logging.getLogger(__name__)


In [69]:
path = os.path.realpath("networks")+'/base.nc'


In [76]:
n = pypsa.Network(path)

n

Index(['380986751-1', '173682341-1', '655093026-1', '383315204-1',
       '655093020-1', '383205136-1', '675799022-1', '380786690-1',
       '611220849-1', '293774847-1',
       ...
       '455897622-1', '456114661-1', '454738664-1', '450529701-1',
       '455897620-1', '456151088-1', '668505779-1', '445315079-1',
       '696726292-1', '696726295-1'],
      dtype='object', name='name', length=9948)
Index(['380986751-1', '173682341-1', '655093026-1', '383315204-1',
       '655093020-1', '383205136-1', '675799022-1', '380786690-1',
       '611220849-1', '293774847-1',
       ...
       '455897622-1', '456114661-1', '454738664-1', '450529701-1',
       '455897620-1', '456151088-1', '668505779-1', '445315079-1',
       '696726292-1', '696726295-1'],
      dtype='object', name='name', length=9948)
INFO:pypsa.io:Imported network base.nc has buses, lines


Network PyPSA-Eur

In [77]:
n.buses

,v_nom,symbol,under_construction,tag_substation,tag_area,lon,lat,geometry,country,type,x,y,carrier,v_mag_pu_set,v_mag_pu_min,v_mag_pu_max,control,sub_network
name,,,,,,,,,,,,,,,,,,
221085150-1,380.0,substation,True,transmission,241710.0,3.574875,36.250907,POINT (3.574875141196995 36.25090712659074),algeria,,0.0,0.0,AC,1.0,0.0,inf,PQ,
221085150-2,220.0,substation,True,transmission,241710.0,3.574875,36.250907,POINT (3.574875141196995 36.25090712659074),algeria,,0.0,0.0,AC,1.0,0.0,inf,PQ,
751040071-1,380.0,substation,True,transmission,120930.0,-0.168282,35.766294,POINT (-0.168282199999985 35.76629409999999),algeria,,0.0,0.0,AC,1.0,0.0,inf,PQ,
751040071-2,220.0,substation,True,transmission,120930.0,-0.168282,35.766294,POINT (-0.168282199999985 35.76629409999999),algeria,,0.0,0.0,AC,1.0,0.0,inf,PQ,
383755661-1,220.0,substation,True,transmission,26420.0,6.021865,31.760655,POINT (6.021864684247905 31.7606548606319),algeria,,0.0,0.0,AC,1.0,0.0,inf,PQ,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
678605405-1,220.0,substation,True,transmission,8150.0,30.160076,-16.900288,POINT (30.16007555283895 -16.90028761222456),zimbabwe,,0.0,0.0,AC,1.0,0.0,inf,PQ,
240035191-1,220.0,substation,True,transmission,7830.0,30.940673,-17.782664,POINT (30.94067330533207 -17.7826636186806),zimbabwe,,0.0,0.0,AC,1.0,0.0,inf,PQ,
409152931-1,220.0,substation,True,transmission,11120.0,31.085101,-17.748434,POINT (31.08510051345972 -17.74843403724154),zimbabwe,,0.0,0.0,AC,1.0,0.0,inf,PQ,


In [80]:
pd.set_option('display.max_columns', None)
n.lines


,v_nom,num_parallel,length,underground,under_construction,tag_type,tag_frequency,geometry,country,bus0,bus1,type,x,r,g,b,s_nom,s_nom_extendable,s_nom_min,s_nom_max,s_max_pu,capital_cost,terrain_factor,v_ang_min,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
380986751-1,220.0,1.0,40.467296,False,False,line,50,"LINESTRING (-1.8363817 34.78690679999992, -1.8...",algeria,,,,0.0,0.0,0.0,0.0,0.0,False,0.0,inf,1.0,0.0,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
173682341-1,380.0,1.0,0.168068,False,False,line,50,"LINESTRING (-0.9017649 35.08983150000002, -0.9...",algeria,,,,0.0,0.0,0.0,0.0,0.0,False,0.0,inf,1.0,0.0,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
655093026-1,220.0,1.0,0.064988,False,False,line,50,"LINESTRING (0.1462109 34.90904640000009, 0.146...",algeria,,,,0.0,0.0,0.0,0.0,0.0,False,0.0,inf,1.0,0.0,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
383315204-1,220.0,1.0,98.637025,False,False,line,50,"LINESTRING (3.3801723 36.71033960000008, 3.383...",algeria,,,,0.0,0.0,0.0,0.0,0.0,False,0.0,inf,1.0,0.0,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
655093020-1,220.0,1.0,0.110799,False,False,line,50,"LINESTRING (0.1466111 34.90939410000009, 0.146...",algeria,,,,0.0,0.0,0.0,0.0,0.0,False,0.0,inf,1.0,0.0,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456151088-1,220.0,1.0,0.131901,True,False,cable,50,LINESTRING (31.01196310000006 -29.813610599999...,south-africa,,,,0.0,0.0,0.0,0.0,0.0,False,0.0,inf,1.0,0.0,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
668505779-1,220.0,1.0,36.903154,True,False,cable,50,"LINESTRING (39.19949770000008 -6.6373292, 39.1...",tanzania,,,,0.0,0.0,0.0,0.0,0.0,False,0.0,inf,1.0,0.0,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
445315079-1,220.0,1.0,0.948420,True,False,cable,50,LINESTRING (9.976712599999923 37.2104494000005...,tunisia,,,,0.0,0.0,0.0,0.0,0.0,False,0.0,inf,1.0,0.0,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
